In [ ]:
from pathlib import Path
from datetime import datetime, timedelta
import tempfile
import zipfile
import io
import os
import sys

import pandas as pd
import sqlalchemy
import requests_cache
import requests
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
%matplotlib inline

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import ktp.census

In [ ]:
NAME = 'haringey-scenario-lsoa-age'
SPATIAL_RESOLUTION = ktp.census.GeographicalLayer.LSOA

In [ ]:
PATH_TO_RESULT_FILE = Path('./build/{}-results.db'.format(NAME)).absolute()

LONDON_BOUNDARY_FILE_URL = 'https://files.datapress.com/london/dataset/statistical-gis-boundary-files-london/2016-10-03T13:52:28/statistical-gis-boundaries-london.zip'
LSOA_SHAPE_FILE_PATH = Path('./statistical-gis-boundaries-london/ESRI/LSOA_2011_London_gen_MHW.shp')
BUILD_FOLDER = Path('./build') / NAME
BUILD_FOLDER.mkdir(parents=True, exist_ok=True)

In [ ]:
disk_engine = sqlalchemy.create_engine('sqlite:///{}'.format(PATH_TO_RESULT_FILE))
requests_cache.install_cache((BUILD_FOLDER / 'cache').as_posix())

In [ ]:
def timedelta_from_iso_string(timedelta_as_string):
    t = datetime.strptime(timedelta_as_string,"PT%MM%SS")
    return timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)

assert timedelta_from_iso_string('PT19M43S') == timedelta(minutes=19) + timedelta(seconds=43)

## Read Metadata

In [ ]:
metadata = pd.read_sql_query('SELECT * FROM metadata', disk_engine, index_col='key', parse_dates=True)

In [ ]:
metadata

In [ ]:
metadata = dict(zip(metadata.index, [value[0] for value in metadata.values]))

In [ ]:
simulation_duration = timedelta_from_iso_string(metadata['durationOfSimulation'])

## Dwellings

In [ ]:
dwellings = pd.read_sql_query('SELECT * FROM dwellings', disk_engine, index_col='index')

In [ ]:
dwellings.head()

## People

In [ ]:
people = pd.read_sql_query('SELECT * FROM people', disk_engine, index_col='index')
people.head()

In [ ]:
dwellings['householdSize'] = people.groupby('dwellingId').size()

## Thermal Power

In [ ]:
thermal_power = pd.read_sql_query('SELECT * FROM thermalPower', disk_engine, index_col='timestamp', parse_dates=True)
thermal_power.index = pd.to_datetime(thermal_power.index * 1000 * 1000)
thermal_power.index.name = 'datetime'
thermal_power = thermal_power.pivot(columns='id')
thermal_power.columns = thermal_power.columns.droplevel(0)
thermal_power.name = 'thermal power'

In [ ]:
dwellings['average_power'] = thermal_power.mean()

In [ ]:
ax = thermal_power\
    .groupby(axis=1, by=lambda id: dwellings.loc[id, 'region'])\
    .mean()['2005-01-01']\
    .plot(figsize=(14, 7), legend=None)
_ = plt.ylabel('average thermal power per household [W]')
_ = plt.title('Average of thermal power per household in different LSOA')
fig = ax.get_figure()
fig.savefig((BUILD_FOLDER / 'thermal_power_per_lsoa.png').as_posix())

In [ ]:
r = requests.get(LONDON_BOUNDARY_FILE_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
with tempfile.TemporaryDirectory(prefix='london-boundary-files') as tmpdir:
    z.extractall(path=tmpdir)
    lsoa_file = Path(tmpdir) / LSOA_SHAPE_FILE_PATH
    lsoa_data = gpd.read_file(lsoa_file.as_posix())
lsoa_data = lsoa_data[lsoa_data.LAD11NM == 'Haringey']

In [ ]:
lsoa_data.head()

In [ ]:
import geopandasplotting as gpdplt

In [ ]:
ax = gpdplt.plot_dataframe(
    lsoa_data.join(dwellings.groupby('region').average_power.mean(), on='LSOA11CD'),
    column='average_power',
    categorical=False, 
    linewidth=0.2, 
    legend=True,
    figsize=(14, 7),
    cmap='viridis'
)
_ = plt.title("Average Thermal Power per Household in different LSOAs [W]")
_ = plt.xticks([])
_ = plt.yticks([])
fig = ax.get_figure()
fig.savefig((BUILD_FOLDER / 'thermal_power_lsoa_choropleth.png').as_posix())

In [ ]:
max_power_lsoa = dwellings.groupby('region').average_power.mean().max()
min_power_lsoa = dwellings.groupby('region').average_power.mean().min()
print(max_power_lsoa/min_power_lsoa)

There is a range of roughly 8% difference between the lsoa with the highest energy consumption and the lsoa with the lowest energy consumption.

In [ ]:
fig = plt.figure(figsize=(7, 7))
sns.violinplot(data=dwellings.groupby('region').average_power.mean(), jitter=True)
_ = plt.ylabel('average thermal power per household [W]')
_ = plt.xticks([])
_ = plt.title("Distribution of average thermal power per household among LSOAs")
fig.savefig((BUILD_FOLDER / "distributation-average-power.png").as_posix())

In [ ]:
fig = plt.figure(figsize=(14, 7))
sns.boxplot(data=dwellings, x='householdSize', y='average_power')
_ = plt.ylabel("average thermal power per household [W]")
_ = plt.xlabel("household size")
_ = plt.title("Average thermal power per household for different household sizes")
fig.savefig((BUILD_FOLDER / "thermal-power-vs-household-size.png").as_posix())

In [ ]:
sns.barplot(data=dwellings, x='region', y='householdSize')

In [ ]:
dwellings.groupby('region').householdSize.mean().corr(dwellings.groupby('region').average_power.mean())

In [ ]:
householdTypes = ktp.census.read_household_type_data(SPATIAL_RESOLUTION)
age_structure = ktp.census.read_age_structure_data(SPATIAL_RESOLUTION)
qualification_data = ktp.census.read_qualification_level_data(SPATIAL_RESOLUTION)
economic_activity_data = ktp.census.read_economic_activity_data(SPATIAL_RESOLUTION)

In [ ]:
AGE_MAP = {
    ktp.types.AgeStructure.AGE_0_TO_4: 2.5,
    ktp.types.AgeStructure.AGE_5_TO_7: 6.5,
    ktp.types.AgeStructure.AGE_8_TO_9: 9,
    ktp.types.AgeStructure.AGE_10_TO_14: 12.5,
    ktp.types.AgeStructure.AGE_15: 15.5,
    ktp.types.AgeStructure.AGE_16_TO_17: 17,
    ktp.types.AgeStructure.AGE_18_TO_19: 19,
    ktp.types.AgeStructure.AGE_20_TO_24: 22.5,
    ktp.types.AgeStructure.AGE_25_TO_29: 27.5,
    ktp.types.AgeStructure.AGE_30_TO_44: 37.5,
    ktp.types.AgeStructure.AGE_45_TO_59: 52.5,
    ktp.types.AgeStructure.AGE_60_TO_64: 62.5,
    ktp.types.AgeStructure.AGE_65_TO_74: 70,
    ktp.types.AgeStructure.AGE_75_TO_84: 80,
    ktp.types.AgeStructure.AGE_85_TO_89: 87.5,
    ktp.types.AgeStructure.AGE_90_AND_OVER: 95 # FIXME
}

def meanAge(age_structure):
    age_structure_num = age_structure.copy()
    for col in age_structure:
        age_structure_num[col] = age_structure[col] * AGE_MAP[col]
    return age_structure_num.sum(axis=1) / age_structure.sum(axis=1)
    
    
def percent_highest_qualification(qualification_data):
    return qualification_data[ktp.types.Qualification.LEVEL_45] / qualification_data.sum(axis=1)


def percent_economic_active(economic_activity_data):
    total_active = economic_activity_data[[ktp.types.EconomicActivity.EMPLOYEE_PART_TIME, 
                                           ktp.types.EconomicActivity.EMPLOYEE_FULL_TIME,
                                           ktp.types.EconomicActivity.SELF_EMPLOYED, 
                                           ktp.types.EconomicActivity.ACTIVE_FULL_TIME_STUDENT]].sum(axis=1)
    return total_active / economic_activity_data.sum(axis=1) 

In [ ]:
data = lsoa_data.copy()
data.set_index('LSOA11CD', inplace=True)
data['average_power'] = dwellings.groupby('region').average_power.mean()
data['number_households'] = householdTypes.sum(axis=1)
data['number citizens'] = age_structure.sum(axis=1)
data['avg household size'] = age_structure.sum(axis=1)/householdTypes.sum(axis=1)
data['avg age'] = meanAge(age_structure)
data['percent highest qual'] = percent_highest_qualification(qualification_data)
data['percent economic act'] = percent_economic_active(economic_activity_data)


In [ ]:
fig = sns.pairplot(
    data=data, 
    y_vars=['average_power'], 
    x_vars=['number_households', 'number citizens', 'avg household size', 'avg age', 
            'percent highest qual', 'percent economic act'])
fig.savefig((BUILD_FOLDER / 'pairwise-distributions.png').as_posix())